In [2]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=[ "ner","tagger",'textcat'])
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
import emoji as emoji
import regex
import spacy
from spacy.tokens import Token
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import Panel
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import time
import numpy as np
import warnings
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
warnings.filterwarnings("ignore")

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


In [3]:
from sklearn.neural_network import MLPClassifier

In [4]:
def mlp_grid_search(hyper_comb,X_train,y_train,file_name):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in tqdm(grid):
        activation=hypers['activation']
        alpha=hypers['alpha']
        batch_size=hypers['batch_size']
        beta_1=hypers['beta_1']
        beta_2=hypers['beta_2']
        early_stopping=hypers['early_stopping']
        learning_rate_init=hypers['learning_rate_init']
        learning_rate=hypers['learning_rate']
        max_iter=hypers['max_iter']
        momentum=hypers['momentum']
        warm_start=hypers['warm_start']
        n_iter_no_change=hypers['n_iter_no_change']
        nesterovs_momentum=hypers['nesterovs_momentum']
        power_t=hypers['power_t']
        random_state=hypers['random_state']
        shuffle=hypers['shuffle']
        solver=hypers['solver']
        tol=hypers['tol']
        validation_fraction=hypers['validation_fraction']
        hidden_layer_sizes=hypers['hidden_layer_sizes']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in cv.split(X_train): # cross validation
            
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=MLPClassifier(activation=activation, alpha=alpha, batch_size=batch_size, beta_1=beta_1,
              beta_2=beta_2, early_stopping=early_stopping, epsilon=1e-08,
              hidden_layer_sizes=hidden_layer_sizes, learning_rate=learning_rate,
              learning_rate_init=learning_rate_init, max_iter=max_iter, momentum=momentum,
              n_iter_no_change=n_iter_no_change, nesterovs_momentum=nesterovs_momentum, power_t=power_t,
              random_state=random_state, shuffle=shuffle, solver=solver, tol=tol,
              validation_fraction=validation_fraction, verbose=False, warm_start=warm_start).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)
        print(np.mean(val_f1_list),np.mean(train_f1_list))



        all_res_cv.append({'activation':activation,
                            'alpha':alpha,
                            'batch_size':batch_size,
                            'beta_1':beta_1,
                            'beta_2':beta_2,
                            'early_stopping':early_stopping,
                            'learning_rate_init':learning_rate_init,
                            'max_iter':max_iter,
                            'momentum':momentum,
                            'n_iter_no_change':n_iter_no_change,
                            'nesterovs_momentum':nesterovs_momentum,
                            'power_t':power_t,
                            'random_state':random_state,
                            'shuffle':shuffle,
                            'solver':solver,
                            'tol':tol,
                            'validation_fraction':validation_fraction,
                            'hidden_layer_sizes':hidden_layer_sizes,
                            'learning_rate':learning_rate,
                            'warm_start':warm_start,
                           'val_f1':np.mean(val_f1_list), 
                           'train_f1':np.mean(train_f1_list),
                           'val_recall':np.mean(val_recall_list), 
                           'train_recall':np.mean(train_recall_list),
                           'val_acc':np.mean(val_accuracy_list), 
                           'train_acc':np.mean(train_accuracy_list),
                           'val_pre':np.mean(val_pre_list),
                          'train_pre':np.mean(train_pre_list)})
            #print(np.mean(val_f1_list),np.mean(train_f1_list))
            
            

        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv(file_name, index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [5]:
train=pd.read_csv('pre_train.csv')

In [6]:
sub=list(train['prep_text'])
sub[5115]='Err:509'
train['prep_text']=sub

## bow_models

In [7]:
X = train['prep_text']
y = train['target']

In [8]:
count_vectorizer = feature_extraction.text.CountVectorizer()
X_train_=count_vectorizer.fit_transform(X)

In [9]:
hyper_comb_lbgs={'activation':['identity','logistic','tanh','relu'],
'alpha':list(np.arange(0.0001,0.01,0.001)),
'batch_size':['auto'],
'beta_1':[0.4],
'beta_2':[0],
'early_stopping':[True],
'learning_rate_init':[0.006],
'max_iter':[100,200],
'momentum':[0.1],
'n_iter_no_change':[10],
'nesterovs_momentum':[True],
'power_t':[0.5],
'random_state':[1848773],
'shuffle':[True],
'solver':['lbfgs'],
'tol':[1e-3],
'validation_fraction':[0.15],
'hidden_layer_sizes':[(10,10,10)],
'learning_rate':['constant'],
'warm_start':[True]}

mlp_grid_search(hyper_comb_lbgs,X_train_,y,'mlp_lgbs_grid_search_cv_bow.csv')

In [ ]:
hyper_comb_sgd={'activation':['identity','logistic','tanh','relu'],
'alpha':list(list(np.arange(0.0001,0.01,0.001))),
'batch_size':['auto'],
'beta_1':[0.4],
'beta_2':[0],
'early_stopping':[True,False],
'learning_rate_init':[0.001,0.004,0.002],
'max_iter':[100,200,300],
'momentum':[0.95,0.7,0.9],
'n_iter_no_change':[10],
'nesterovs_momentum':[True,False],
'power_t':[0.6,0.5,0.3],
'random_state':[1848773],
'shuffle':[True,False],
'solver':['sgd'],
'tol':[1e-3],
'validation_fraction':[0.1,0.2],
'hidden_layer_sizes':[(100,)],
'learning_rate':['constant','invscaling','adaptative'],
'warm_start':[True]}

In [10]:
hyper_comb_adam={'activation':['identity','logistic','tanh','relu'],
'alpha':list(list(np.arange(0.0001,0.01,0.005))),
'batch_size':['auto'],
'beta_1':[0.1,0.3,0.5,0.7,0.9],
'beta_2':[0.1,0.3,0.5,0.7,0.9],
'early_stopping':[True],
'learning_rate_init':[0.001],
'max_iter':[200],
'momentum':[0.1],
'n_iter_no_change':[20],
'nesterovs_momentum':[True],
'power_t':[0.5],
'random_state':[1848773],
'shuffle':[True,False],
'solver':['adam'],
'tol':[1e-3],
'validation_fraction':[0.12],
'hidden_layer_sizes':[(10,10,10)],
'learning_rate':['constant'],
'warm_start':[True]}

In [11]:
mlp_grid_search(hyper_comb_adam,X_train_,y,'mlp_adam_grid_search_cv_bow.csv')

  0%|          | 0/400 [00:00<?, ?it/s]

0.7965310702388173 0.8460194386236767


  0%|          | 1/400 [00:15<1:45:10, 15.81s/it]

0.80231146942674 0.8435244225578185


  0%|          | 2/400 [00:28<1:38:37, 14.87s/it]

...

0.8059895444619212 0.907625075517217


100%|█████████▉| 399/400 [1:54:07<00:14, 14.88s/it]

0.8045443368136234 0.9093324426278077


100%|██████████| 400/400 [1:54:22<00:00, 17.16s/it]


File created grid_search_cv


In [14]:
max(list(pd.read_csv('mlp_adam_grid_search_cv_bow.csv')['val_f1']))

0.8074345795481115

### tf_idf

In [16]:
tfidf_transformer = TfidfTransformer()
train_tfidf_ = tfidf_transformer.fit_transform(X_train_)

In [ ]:
hyper_comb={'alpha':list(np.arange(0,35,0.25)),
            'fit_prior':[True,False]} ##combination of hyperparameters

mlp_grid_search(hyper_comb_lbgs,X_train_,y,'mlp_lgbs_grid_search_cv_tfidf.csv')

In [17]:
mlp_grid_search(hyper_comb_adam,train_tfidf_,y,'mlp_adam_grid_search_cv_tfidf.csv')

  0%|          | 0/400 [00:00<?, ?it/s]

0.7941661928398807 0.8449357702648314


  0%|          | 1/400 [00:14<1:35:09, 14.31s/it]

0.798765490684666 0.8404703971160983


  0%|          | 2/400 [00:26<1:30:31, 13.65s/it]

...

0.8004715259580865 0.8983982990328135


100%|█████████▉| 399/400 [1:40:10<00:21, 21.20s/it]

0.8009973226989058 0.8999741954198217


100%|██████████| 400/400 [1:40:32<00:00, 15.08s/it]


File created grid_search_cv


MultinomialNB_grid_search(hyper_comb,train_tfidf_,y,'multinomial_grid_search_cv_tfidf.csv')

In [18]:
max(list(pd.read_csv('mlp_adam_grid_search_cv_tfidf.csv')['val_f1']))

0.8049387275097649